<a href="https://colab.research.google.com/github/Fjoru/TM10007_PROJECT/blob/Erik/assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TM10007 Assignment Prediction of tumor grade in brain cancer
By Jessica Barends, Gonnie van Erp, Erik Kemper en Carlijn Oerlemans

In [1]:
# Run install for use in colab environment
!pip install --upgrade pip
!pip install -q --upgrade git+https://github.com/Fjoru/TM10007_PROJECT
!pip install ipdb -q
!pip install seaborn
!pip install tensorflow

     |████████████████████████████████| 1.4MB 3.4MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1


## Import section


In [0]:
import ipdb
import pandas as pd
import seaborn as sns
import numpy as np
# import tensorflow as tf

# Classifiers
from sklearn import preprocessing
from sklearn import model_selection
from sklearn import metrics

## preprocessing

In [0]:
def Replace(i):
    if isinstance(i, int):
          return i
    try:
        float(i)
        return float(i)
    except:
        return np.nan

def preprocessing_steps(data):
    min_max_scaler = preprocessing.MinMaxScaler()
    # split labels and values
    data_X = data.drop(columns=['label'])
    data_Y = data[['label']]
    
    # remove strings from data
    data_X = data_X.applymap(func=Replace)
 

    # remove features with less than 80% values
    data_X = data_X.dropna(thresh=round(len(data_X)*0.8), axis='columns')

    # Add missing value's 
    pd.options.mode.use_inf_as_na = True # inf waardes zien als nan
    data_X = data_X.fillna(0)

    # normalisation of values
    data_X = pd.DataFrame(preprocessing.normalize(data_X.T, norm='l2'), columns=data_X.index, index=data_X.columns )
    data_X = data_X.T

    return data_X, data_Y

## Run Pipeline

run all predefined steps

In [6]:
# Data loading functions.
from brats.load_data import load_data

data = load_data()
print(f'The number of samples: {len(data.index)}')
print(f'The number of columns: {len(data.columns)}')
data = pd.DataFrame(data)

# data split index forming
Test_split = model_selection.StratifiedKFold(n_splits=10)

# run preprocessing step
data_X, data_Y = preprocessing_steps(data)

# save data to csv for manual check
data_X.to_csv('data_X.csv')
data_Y.to_csv('data_Y.csv')

# change data to numpy format
data_X = np.array(data_X)
data_Y = np.array(data_Y)

for design_index, test_index in Test_split.split(data_X, data_Y):
    X_design = data_X[design_index]
    Y_design = data_Y[design_index]
    
    X_test = data_X[test_index]
    Y_test = data_Y[test_index]



The number of samples: 167
The number of columns: 725
